Script para tratamento dos dados para OD por geolocalização

Pacotes

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

Carregamento dos arquivos



In [ ]:

#Arquivo bilhetagem
sbe_caminho = r"G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_ADAPTACAOBOTELHO\Dezembro_2023\20231206\exputiliz_2023-12-06.csv"
sbe = pd.read_csv(sbe_caminho, sep=";", encoding="latin1", dtype={'CODIGO_VEICULO': 'object', 'CODIGO_LINHA': 'object'}, low_memory=False)

In [ ]:
sbe['DATAHORA_UTILIZACAO'] = pd.to_datetime(sbe['DATAHORA_UTILIZACAO'])

In [ ]:
sbe['DATAHORA_UTILIZACAO'] = pd.to_datetime(sbe['DATAHORA_UTILIZACAO'])
sbe['HORARIO'] = sbe['DATAHORA_UTILIZACAO'].dt.strftime('%H:%M:%S')
sbe['DATA'] = sbe['DATAHORA_UTILIZACAO'].dt.strftime('%Y-%m-%d')
sbe['HORARIO'] = pd.to_datetime(sbe['HORARIO'])

In [ ]:
print(sbe[sbe['DATA'] == "2023-12-07"])

In [ ]:
valores_unicos = sbe['CODIGO_VEICULO'].unique()
valores_unicos

In [ ]:
sbe= sbe[
    ~(
        (sbe['TIPO_CARTAO'] == 'UNITÁRIO BRT') |(sbe['TIPO_CARTAO'] == 'CARTAO VIRTUAL')| (sbe['TIPO_CARTAO'] == 'OPERACIONAL BHTRANS') | (sbe['TIPO_CARTAO'] == 'ADMINISTRATIVO') | (sbe['TIPO_CARTAO'] == 'ACESSO TECNICO E OPERACIONAL') | (sbe['TIPO_CARTAO'] == 'GRATUIDADE SEM CARTAO') | (sbe['TIPO_CARTAO'] == 'AUXILIAR DE OPERAÇÕES')| (sbe['TIPO_CARTAO'] == 'RODOVIÁRIO METROPOLITANO'))]

In [ ]:
sbe

Associando o gps PING com o sbe

In [ ]:
# Aplicando múltiplos filtros em uma única operação
sbe_ping = sbe[
    ~(
        (sbe['NOME_OPERADORA'] == 'METRÔ BH') |
        (sbe['NOME_OPERADORA'] == 'CBTU') |
        sbe['NOME_OPERADORA'].str.startswith('ESTAÇÃO') |
        (sbe['NOME_OPERADORA'] == 'SINDPAUTRAS') |
        (sbe['CODIGO_LINHA'].str.startswith("AC")) |
        (sbe['CODIGO_LINHA'].str.startswith('CM')) |
        (sbe['CODIGO_LINHA'].str.startswith('PR')) |
        (sbe['CODIGO_LINHA'].str.startswith('SD')) |
        (sbe['CODIGO_VEICULO'].str.startswith('UC')) |
        (sbe['CODIGO_VEICULO'].str.startswith('UE'))
    )
]


In [ ]:
print(sbe_ping)

In [ ]:
def processar_linha(linha):
    # Remove caracteres indesejados e separa com base em ';'
    partes = linha.strip('<>;\n').split(';')
    # Cria um dicionário a partir dos valores separados
    return {parte.split('=')[0]: parte.split('=')[1] for parte in partes if parte}

diretorio_atual = os.getcwd()
caminho_pasta = os.path.join(diretorio_atual, "PING")

# Lista para armazenar os DataFrames
dfs = []

for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith(".csv"):
        caminho_completo = os.path.join(caminho_pasta, arquivo)
        with open(caminho_completo, 'r', encoding='latin-1') as f:
            linhas = f.readlines()[1:]  # Pula o cabeçalho
            dados = [processar_linha(linha) for linha in linhas]
            df = pd.DataFrame(dados)
            dfs.append(df)

gps = pd.concat(dfs, ignore_index=True)

In [60]:
gps = gps.drop_duplicates().reset_index(drop=True)
gps["geometry"] = gps.apply(lambda row: Point(row["LG"], row["LT"]), axis=1)

gps_gdf = gpd.GeoDataFrame(gps, geometry="geometry", crs="EPSG:4326")
# gps_gdf = gps_gdf.to_crs("EPSG:31983")
gps_gdf["HR"] = pd.to_datetime(gps_gdf["HR"], format="%Y%m%d%H%M%S")
gps_gdf = gps_gdf.sort_values(by=["HR"]).reset_index(drop=True)


In [61]:
gps_gdf.rename(columns={"HR": "DH", "NV": "CODIGO_VEICULO"}, inplace=True)

In [62]:
gps_gdf['DH'] = pd.to_datetime(gps_gdf['DH'])
gps_gdf['HORARIO_GPS'] = gps_gdf['DH'].dt.strftime('%H:%M:%S')
gps_gdf['DATA_GPS'] = gps_gdf['DH'].dt.strftime('%Y-%m-%d')
gps_gdf = gps_gdf.dropna().reset_index(drop=True)

gps_gdf

,EV,LG,LT,CODIGO_VEICULO,DH,geometry,HORARIO_GPS,DATA_GPS
0,105,-43.927544,-19.851815,20982,2023-12-06 00:00:00,POINT (-43.92754 -19.85181),00:00:00,2023-12-06
1,105,-43.935909,-19.838122,10909,2023-12-06 00:00:00,POINT (-43.93591 -19.83812),00:00:00,2023-12-06
2,105,-43.969893,-19.810253,30602,2023-12-06 00:00:00,POINT (-43.96989 -19.81025),00:00:00,2023-12-06
3,105,-44.009780,-19.945610,40318,2023-12-06 00:00:00,POINT (-44.00978 -19.94561),00:00:00,2023-12-06
4,105,-44.022164,-19.897082,11316,2023-12-06 00:00:00,POINT (-44.02216 -19.89708),00:00:00,2023-12-06
...,...,...,...,...,...,...,...,...
5452268,105,-44.034943,-19.988216,31072,2023-12-07 02:00:00,POINT (-44.03494 -19.98822),02:00:00,2023-12-07
5452269,105,-43.969978,-19.952955,30999,2023-12-07 02:00:00,POINT (-43.96998 -19.95295),02:00:00,2023-12-07
5452270,105,-43.969927,-19.810253,10683,2023-12-07 02:00:00,POINT (-43.96993 -19.81025),02:00:00,2023-12-07
5452271,105,-43.935871,-19.838387,10940,2023-12-07 02:00:00,POINT (-43.93587 -19.83839),02:00:00,2023-12-07


In [63]:
gps_gdf["CODIGO_VEICULO"]= gps_gdf["CODIGO_VEICULO"].astype("int64")
sbe_ping["CODIGO_VEICULO"]= sbe_ping["CODIGO_VEICULO"].astype("int64")

D:\usuarios\sm00000028\AppData\Local\Temp\ipykernel_4256\2041716247.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sbe_ping["CODIGO_VEICULO"]= sbe_ping["CODIGO_VEICULO"].astype("int64")


In [64]:


# Convertendo as colunas de horário para datetime, se ainda não estiverem
gps_gdf['HORARIO_GPS'] = pd.to_datetime(gps_gdf['HORARIO_GPS'])
sbe_ping['HORARIO'] = pd.to_datetime(sbe_ping['HORARIO'])



# Fazendo o merge usando merge_asof para encontrar as linhas correspondentes em gps_gdf
merged = pd.merge_asof(sbe_ping.sort_values('HORARIO'),
                       gps_gdf.sort_values('HORARIO_GPS'),
                       left_on='HORARIO',
                       right_on='HORARIO_GPS',
                       by='CODIGO_VEICULO',
                       direction='backward',
                       tolerance=pd.Timedelta(seconds=40))


# Resultado contém as geometrias do gps_gdf correspondentes ao horário mais próximo anterior em sbe_final


D:\usuarios\sm00000028\AppData\Local\Temp\ipykernel_4256\671619087.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gps_gdf['HORARIO_GPS'] = pd.to_datetime(gps_gdf['HORARIO_GPS'])
D:\usuarios\sm00000028\AppData\Local\Temp\ipykernel_4256\671619087.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sbe_ping['HORARIO'] = pd.to_datetime(sbe_ping['HORARIO'])


Associando o metrô com a localização

In [65]:
# Filtrando as linhas com NOME_OPERADORA igual a 'METRÔ BH'
sbe_metro = sbe[(sbe['NOME_OPERADORA'] == 'METRÔ BH') | (sbe['NOME_OPERADORA'] == 'CBTU')]



sbe_metro



,NOME_OPERADORA,CODIGO_VEICULO,CODIGO_LINHA,CARTAO_USUARIO,TIPO_CARTAO,DATAHORA_UTILIZACAO,VALOR_COBRADO,Unnamed: 7,HORARIO,DATA
0,METRÔ BH,UCI10,5002,6850003814588,VT CARGA A BORDO,2023-12-06 05:16:21,530,NaN,2024-02-16 05:16:21,2023-12-06
1,METRÔ BH,UCI10,5002,6700003137276,CARTÃO AO PORTADOR,2023-12-06 05:27:02,530,NaN,2024-02-16 05:27:02,2023-12-06
2,METRÔ BH,UCI10,5002,6850000646767,VT CARGA A BORDO,2023-12-06 05:29:50,530,NaN,2024-02-16 05:29:50,2023-12-06
3,METRÔ BH,UCI10,5002,6850003580417,VT CARGA A BORDO,2023-12-06 05:48:55,530,NaN,2024-02-16 05:48:55,2023-12-06
4,METRÔ BH,UCI10,5002,6700002581664,CARTÃO AO PORTADOR,2023-12-06 05:49:11,530,NaN,2024-02-16 05:49:11,2023-12-06
...,...,...,...,...,...,...,...,...,...,...
42101,METRÔ BH,UWL12,5017,6850003686137,VT CARGA A BORDO,2023-12-06 21:13:45,530,NaN,2024-02-16 21:13:45,2023-12-06
42102,METRÔ BH,UWL12,5017,6850003291720,VT CARGA A BORDO,2023-12-06 21:52:49,530,NaN,2024-02-16 21:52:49,2023-12-06
42103,METRÔ BH,UWL12,5017,6700003024089,CARTÃO AO PORTADOR,2023-12-06 21:54:05,530,NaN,2024-02-16 21:54:05,2023-12-06
42104,METRÔ BH,UWL12,5017,6700003118112,CARTÃO AO PORTADOR,2023-12-06 22:18:38,530,NaN,2024-02-16 22:18:38,2023-12-06


In [66]:
gps_geo_metro_caminho = r"G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_HEXAGONO\ESTACAO_METRO.csv"
gps_metro = pd.read_csv(gps_geo_metro_caminho, sep= ";", encoding= "latin1")
gps_metro = gps_metro.rename(columns={'CODIGO': 'CODIGO_VEICULO'})
# Converta a coluna 'CODIGO_LINHA' para string em ambos os DataFrames
sbe_metro['CODIGO_LINHA'] = sbe_metro['CODIGO_LINHA'].astype(str)
gps_metro['CODIGO_LINHA'] = gps_metro['CODIGO_LINHA'].astype(str)

# Realize um merge entre os DataFrames usando a coluna 'CODIGO_LINHA' como chave
sbe_metro_com_geometria = pd.merge(sbe_metro, gps_metro[['CODIGO_LINHA', 'geometry']], on='CODIGO_LINHA', how='left')

# O resultado será um novo DataFrame sbe_metro_com_geometria que incluirá a geometria do gps_metro



# # Exibindo o DataFrame resultante
sbe_metro_com_geometria.columns

D:\usuarios\sm00000028\AppData\Local\Temp\ipykernel_4256\4045681569.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sbe_metro['CODIGO_LINHA'] = sbe_metro['CODIGO_LINHA'].astype(str)


Index(['NOME_OPERADORA', 'CODIGO_VEICULO', 'CODIGO_LINHA', 'CARTAO_USUARIO',
       'TIPO_CARTAO', 'DATAHORA_UTILIZACAO', 'VALOR_COBRADO', 'Unnamed: 7',
       'HORARIO', 'DATA', 'geometry'],
      dtype='object')

Associando as estações com as localizações

In [67]:

sbe_estacoes = sbe[sbe['CODIGO_LINHA'].str.startswith(('CM', 'PR', 'SD', 'AC', '1000', '2000', '3000', '4000', '6000','7000'))]



sbe_estacoes


,NOME_OPERADORA,CODIGO_VEICULO,CODIGO_LINHA,CARTAO_USUARIO,TIPO_CARTAO,DATAHORA_UTILIZACAO,VALOR_COBRADO,Unnamed: 7,HORARIO,DATA
56383,ESTAÇÃO BHBUS DIAMANTE,09301,1000,6700003014662,CARTÃO AO PORTADOR,2023-12-06 00:18:09,420,NaN,2024-02-16 00:18:09,2023-12-06
56384,ESTAÇÃO BHBUS DIAMANTE,09301,1000,6850003797371,VT CARGA A BORDO,2023-12-06 00:54:16,420,NaN,2024-02-16 00:54:16,2023-12-06
56385,ESTAÇÃO BHBUS DIAMANTE,09301,1000,6700003044687,CARTÃO AO PORTADOR,2023-12-06 03:15:00,420,NaN,2024-02-16 03:15:00,2023-12-06
56386,ESTAÇÃO BHBUS DIAMANTE,09301,1000,2320000922433,IDOSO,2023-12-06 04:01:09,0,NaN,2024-02-16 04:01:09,2023-12-06
56387,ESTAÇÃO BHBUS DIAMANTE,09301,1000,6700003087396,CARTÃO AO PORTADOR,2023-12-06 04:32:26,420,NaN,2024-02-16 04:32:26,2023-12-06
...,...,...,...,...,...,...,...,...,...,...
964821,ESTAÇÃO BHBUS VENDA NOVA,900113,2000,6700002686829,CARTÃO AO PORTADOR,2023-12-06 20:24:57,420,NaN,2024-02-16 20:24:57,2023-12-06
964828,ESTAÇÃO BHBUS VENDA NOVA,900113,2000,6850003296821,VT CARGA A BORDO,2023-12-06 21:06:01,420,NaN,2024-02-16 21:06:01,2023-12-06
964829,ESTAÇÃO BHBUS VENDA NOVA,900113,2000,2320000919654,IDOSO,2023-12-06 21:17:32,0,NaN,2024-02-16 21:17:32,2023-12-06
964830,ESTAÇÃO BHBUS VENDA NOVA,900113,2000,6700002490100,CARTÃO AO PORTADOR,2023-12-06 21:22:26,420,NaN,2024-02-16 21:22:26,2023-12-06


In [68]:
sbe_est_caminho = r"G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_HEXAGONO\ESTACAO_ONIBUS V2.xlsx"
sbe_est = pd.read_excel(sbe_est_caminho)
sbe_est

,geometry,codigo,CODIGO_LINHA,NOME,TIPO,CORREDOR_MOVE,EIXO_MOVE,SISTEMA_TRONCAL,INTEGRACAO_FISICA_METRO,SITUACAO,GESTOR
0,Point (602517.50643187004607171 7791145.505910...,2,4002,BARREIRO,INTEGRACAO BHBUS,NaN,NaN,CONVENCIONAL,NAO,EM OPERACAO,BHTRANS
1,Point (602101.05831154994666576 7788852.602624...,3,1000,DIAMANTE,INTEGRACAO BHBUS,NaN,NaN,CONVENCIONAL,NAO,EM OPERACAO,BHTRANS/SETOP
2,Point (608066.40717149002011865 7805630.617508...,6,6001,PAMPULHA,INTEGRACAO BHBUS/MOVE,NaN,NaN,MOVE,SIM,EM OPERACAO,BHTRANS
3,Point (612401.75051438994705677 7803290.763778...,7,3001,SAO GABRIEL,INTEGRACAO BHBUS/MOVE/METROPOLITANO,NaN,NaN,MOVE/METRO,SIM,EM OPERACAO,BHBUS/MOVE: BHTRANS - SETOP/METROPOLITANO: SETOP
4,Point (613794.71274545998312533 7800983.363443...,8,BHBUS,JOSE CANDIDO DA SILVEIRA,INTEGRACAO BHBUS,NaN,NaN,METRO,SIM,EM OPERACAO,BHTRANS
...,...,...,...,...,...,...,...,...,...,...,...
58,Point (612401.75051438994705677 7803290.763778...,7,3000,SAO GABRIEL,INTEGRACAO BHBUS/MOVE/METROPOLITANO,NaN,NaN,MOVE/METRO,SIM,EM OPERACAO,BHBUS/MOVE: BHTRANS - SETOP/METROPOLITANO: SETOP
59,Point (612401.75051438994705677 7803290.763778...,7,3002,SAO GABRIEL,INTEGRACAO BHBUS/MOVE/METROPOLITANO,NaN,NaN,MOVE/METRO,SIM,EM OPERACAO,BHBUS/MOVE: BHTRANS - SETOP/METROPOLITANO: SETOP
60,Point (602517.50643187004607171 7791145.505910...,2,4001,BARREIRO,INTEGRACAO BHBUS,NaN,NaN,CONVENCIONAL,NAO,EM OPERACAO,BHTRANS
61,Point (608066.40717149002011865 7805630.617508...,6,6000,PAMPULHA,INTEGRACAO BHBUS/MOVE,NaN,NaN,MOVE,SIM,EM OPERACAO,BHTRANS


In [69]:
sbe_estacoes['CODIGO_LINHA'] = sbe_estacoes['CODIGO_LINHA'].astype(str)
sbe_est['CODIGO_LINHA'] = sbe_est['CODIGO_LINHA'].astype(str)
sbe_est_com_geometria = pd.merge(sbe_estacoes, sbe_est[['CODIGO_LINHA', 'geometry']], on='CODIGO_LINHA', how='left')
sbe_est_com_geometria

D:\usuarios\sm00000028\AppData\Local\Temp\ipykernel_4256\1577255457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sbe_estacoes['CODIGO_LINHA'] = sbe_estacoes['CODIGO_LINHA'].astype(str)


,NOME_OPERADORA,CODIGO_VEICULO,CODIGO_LINHA,CARTAO_USUARIO,TIPO_CARTAO,DATAHORA_UTILIZACAO,VALOR_COBRADO,Unnamed: 7,HORARIO,DATA,geometry
0,ESTAÇÃO BHBUS DIAMANTE,09301,1000,6700003014662,CARTÃO AO PORTADOR,2023-12-06 00:18:09,420,NaN,2024-02-16 00:18:09,2023-12-06,Point (602101.05831154994666576 7788852.602624...
1,ESTAÇÃO BHBUS DIAMANTE,09301,1000,6850003797371,VT CARGA A BORDO,2023-12-06 00:54:16,420,NaN,2024-02-16 00:54:16,2023-12-06,Point (602101.05831154994666576 7788852.602624...
2,ESTAÇÃO BHBUS DIAMANTE,09301,1000,6700003044687,CARTÃO AO PORTADOR,2023-12-06 03:15:00,420,NaN,2024-02-16 03:15:00,2023-12-06,Point (602101.05831154994666576 7788852.602624...
3,ESTAÇÃO BHBUS DIAMANTE,09301,1000,2320000922433,IDOSO,2023-12-06 04:01:09,0,NaN,2024-02-16 04:01:09,2023-12-06,Point (602101.05831154994666576 7788852.602624...
4,ESTAÇÃO BHBUS DIAMANTE,09301,1000,6700003087396,CARTÃO AO PORTADOR,2023-12-06 04:32:26,420,NaN,2024-02-16 04:32:26,2023-12-06,Point (602101.05831154994666576 7788852.602624...
...,...,...,...,...,...,...,...,...,...,...,...
88530,ESTAÇÃO BHBUS VENDA NOVA,900113,2000,6700002686829,CARTÃO AO PORTADOR,2023-12-06 20:24:57,420,NaN,2024-02-16 20:24:57,2023-12-06,Point (607896.77366264001466334 7809269.676788...
88531,ESTAÇÃO BHBUS VENDA NOVA,900113,2000,6850003296821,VT CARGA A BORDO,2023-12-06 21:06:01,420,NaN,2024-02-16 21:06:01,2023-12-06,Point (607896.77366264001466334 7809269.676788...
88532,ESTAÇÃO BHBUS VENDA NOVA,900113,2000,2320000919654,IDOSO,2023-12-06 21:17:32,0,NaN,2024-02-16 21:17:32,2023-12-06,Point (607896.77366264001466334 7809269.676788...
88533,ESTAÇÃO BHBUS VENDA NOVA,900113,2000,6700002490100,CARTÃO AO PORTADOR,2023-12-06 21:22:26,420,NaN,2024-02-16 21:22:26,2023-12-06,Point (607896.77366264001466334 7809269.676788...


In [70]:
sbe_est_com_geometria.columns

Index(['NOME_OPERADORA', 'CODIGO_VEICULO', 'CODIGO_LINHA', 'CARTAO_USUARIO',
       'TIPO_CARTAO', 'DATAHORA_UTILIZACAO', 'VALOR_COBRADO', 'Unnamed: 7',
       'HORARIO', 'DATA', 'geometry'],
      dtype='object')

In [71]:
merged.columns

Index(['NOME_OPERADORA', 'CODIGO_VEICULO', 'CODIGO_LINHA', 'CARTAO_USUARIO',
       'TIPO_CARTAO', 'DATAHORA_UTILIZACAO', 'VALOR_COBRADO', 'Unnamed: 7',
       'HORARIO', 'DATA', 'EV', 'LG', 'LT', 'DH', 'geometry', 'HORARIO_GPS',
       'DATA_GPS'],
      dtype='object')

In [72]:
sbe_metro_com_geometria.columns

Index(['NOME_OPERADORA', 'CODIGO_VEICULO', 'CODIGO_LINHA', 'CARTAO_USUARIO',
       'TIPO_CARTAO', 'DATAHORA_UTILIZACAO', 'VALOR_COBRADO', 'Unnamed: 7',
       'HORARIO', 'DATA', 'geometry'],
      dtype='object')

In [73]:
sbe_unificado = pd.concat([sbe_est_com_geometria, sbe_metro_com_geometria, merged])


In [74]:


# Convertendo 'DATAHORA_UTILIZACAO' para datetime
sbe_unificado['DATAHORA_UTILIZACAO'] = pd.to_datetime(sbe_unificado['DATAHORA_UTILIZACAO'])

# Ordenando o DataFrame
sbe_sorted = sbe_unificado.sort_values(['CARTAO_USUARIO', 'DATAHORA_UTILIZACAO'])

# Calculando a diferença de tempo entre registros consecutivos em minutos
sbe_sorted['time_diff'] = sbe_sorted.groupby('CARTAO_USUARIO')['DATAHORA_UTILIZACAO'].diff().dt.total_seconds() / 60

# Filtrar validações com intervalos menores que 5 minutos
sbe_filtered = sbe_sorted[(sbe_sorted['time_diff'] >= 5) | (sbe_sorted.groupby('CARTAO_USUARIO').cumcount() == 0)]

# Selecionar apenas as primeiras validações de cada cartão
primeiras_val = sbe_filtered[sbe_filtered.groupby('CARTAO_USUARIO').cumcount() == 0]

# Separar as validações com intervalos maiores e menores que 90 minutos
val_maiores_90 = sbe_filtered[sbe_filtered['time_diff'] >= 90]
val_menores_90 = sbe_filtered[sbe_filtered['time_diff'] < 90]

sbe_final = pd.concat([primeiras_val, val_maiores_90, val_menores_90])


In [75]:
print(sbe_final[sbe_final['CARTAO_USUARIO'] == 2310000029300])

                       NOME_OPERADORA CODIGO_VEICULO CODIGO_LINHA  \
57656           URCA AUTO ÔNIBUS LTDA         402001         AC20   
467336          URCA AUTO ÔNIBUS LTDA          40969         4205   
522895  RODOPASS TRANSP.COLET.PASSAG.          10703           51   

        CARTAO_USUARIO TIPO_CARTAO DATAHORA_UTILIZACAO  VALOR_COBRADO  \
57656    2310000029300  DEFICIENTE 2023-12-06 15:09:12              0   
467336   2310000029300  DEFICIENTE 2023-12-06 16:48:33              0   
522895   2310000029300  DEFICIENTE 2023-12-06 17:46:10              0   

        Unnamed: 7             HORARIO        DATA  \
57656          NaN 2024-02-16 15:09:12  2023-12-06   
467336         NaN 2024-02-16 16:48:33  2023-12-06   
522895         NaN 2024-02-16 17:46:10  2023-12-06   

                                                 geometry   EV          LG  \
57656   Point (609918.99276361998636276 7800432.625544...  NaN         NaN   
467336                      POINT (-43.950544 -19.921496)

In [76]:

# Supondo que merged é o seu DataFrame
# Garantir que os dados estão ordenados corretamente
sbe_final = sbe_final.sort_values(['CARTAO_USUARIO', 'DATAHORA_UTILIZACAO'])

# Deslocando os dados para obter a próxima validação
sbe_final['DESTINO_HORARIO'] = sbe_final.groupby('CARTAO_USUARIO')['DATAHORA_UTILIZACAO'].shift(-1)
sbe_final['DESTINO_GEOMETRY'] = sbe_final.groupby('CARTAO_USUARIO')['geometry'].shift(-1)

# Para a última linha de cada grupo, definir o destino como a primeira origem do grupo
for _, group in sbe_final.groupby('CARTAO_USUARIO'):
    if len(group) > 1:
        last_index = group.index[-1]
        first_index = group.index[0]
        sbe_final.at[last_index, 'DESTINO_HORARIO'] = group.at[first_index, 'DATAHORA_UTILIZACAO']
        sbe_final.at[last_index, 'DESTINO_GEOMETRY'] = group.at[first_index, 'geometry']


# Exibindo o resultado
print(sbe_final[sbe_final['CARTAO_USUARIO'] == 6850001298748])


                    NOME_OPERADORA CODIGO_VEICULO CODIGO_LINHA  \
20076   AUTO OMNIBUS FLORAMAR LTDA          10766         5201   
392639       VIA BH COLETIVOS LTDA          31139         9250   
410227  VIAÇÃO GETÚLIO VARGAS LTDA          20695         8102   

        CARTAO_USUARIO       TIPO_CARTAO DATAHORA_UTILIZACAO  VALOR_COBRADO  \
20076    6850001298748  VT CARGA A BORDO 2023-12-06 05:19:34            450   
392639   6850001298748  VT CARGA A BORDO 2023-12-06 14:36:05            450   
410227   6850001298748  VT CARGA A BORDO 2023-12-06 15:16:33            225   

        Unnamed: 7             HORARIO        DATA  \
20076          NaN 2024-02-16 05:19:34  2023-12-06   
392639         NaN 2024-02-16 14:36:05  2023-12-06   
410227         NaN 2024-02-16 15:16:33  2023-12-06   

                             geometry   EV          LG          LT  \
20076   POINT (-43.935184 -19.858585)  105  -43.935184  -19.858585   
392639  POINT (-43.960678 -19.966806)  105  -43.960678  -19.9

Juntando todos os dataframes com as localizações e analisando as diferenças entre 90 minutos de validação

In [77]:
print(sbe_final.columns)

Index(['NOME_OPERADORA', 'CODIGO_VEICULO', 'CODIGO_LINHA', 'CARTAO_USUARIO',
       'TIPO_CARTAO', 'DATAHORA_UTILIZACAO', 'VALOR_COBRADO', 'Unnamed: 7',
       'HORARIO', 'DATA', 'geometry', 'EV', 'LG', 'LT', 'DH', 'HORARIO_GPS',
       'DATA_GPS', 'time_diff', 'DESTINO_HORARIO', 'DESTINO_GEOMETRY'],
      dtype='object')


Tratamento dos dados

Função de ajuste dos dados do gps

In [78]:
# Criando a lista de colunas a serem excluídas
colunas_excluir = ['NOME_OPERADORA', 'Unnamed: 7', 'EV', 'DH', 'DESTINO_HORARIO']

# Usando a lista para excluir as colunas do DataFrame
od = sbe_final.drop(colunas_excluir, axis=1)
od.reset_index(drop=True, inplace=True)
# Exibindo o DataFrame resultante
print(od)


       CODIGO_VEICULO CODIGO_LINHA  CARTAO_USUARIO       TIPO_CARTAO  \
0               40805         3054   2310000029191        DEFICIENTE   
1               31111           33   2310000029191        DEFICIENTE   
2               20476           82   2310000029191        DEFICIENTE   
3               30688         3051   2310000029191        DEFICIENTE   
4               31080         3052   2310000029195        DEFICIENTE   
...               ...          ...             ...               ...   
766959          30656         2034   6850003891202  VT CARGA A BORDO   
766960          40407         9214   6850003891202  VT CARGA A BORDO   
766961          40511           62   6850003891205  VT CARGA A BORDO   
766962         300115         3000   6850003891205  VT CARGA A BORDO   
766963          20493         9411   6850003891206  VT CARGA A BORDO   

       DATAHORA_UTILIZACAO  VALOR_COBRADO             HORARIO        DATA  \
0      2023-12-06 05:12:53              0 2024-02-16 05:12

In [79]:
# Renomeando a coluna 'geometry' para 'ORIGIN_GEOMETRY'
od = od.rename(columns={'geometry': 'ORIGIN_GEOMETRY'})

# Exibindo o DataFrame resultante
print(od.columns)


Index(['CODIGO_VEICULO', 'CODIGO_LINHA', 'CARTAO_USUARIO', 'TIPO_CARTAO',
       'DATAHORA_UTILIZACAO', 'VALOR_COBRADO', 'HORARIO', 'DATA',
       'ORIGIN_GEOMETRY', 'LG', 'LT', 'HORARIO_GPS', 'DATA_GPS', 'time_diff',
       'DESTINO_GEOMETRY'],
      dtype='object')


In [80]:
print(od[od["CARTAO_USUARIO"] == 6850001298748])

       CODIGO_VEICULO CODIGO_LINHA  CARTAO_USUARIO       TIPO_CARTAO  \
331069          10766         5201   6850001298748  VT CARGA A BORDO   
331070          31139         9250   6850001298748  VT CARGA A BORDO   
331071          20695         8102   6850001298748  VT CARGA A BORDO   

       DATAHORA_UTILIZACAO  VALOR_COBRADO             HORARIO        DATA  \
331069 2023-12-06 05:19:34            450 2024-02-16 05:19:34  2023-12-06   
331070 2023-12-06 14:36:05            450 2024-02-16 14:36:05  2023-12-06   
331071 2023-12-06 15:16:33            225 2024-02-16 15:16:33  2023-12-06   

                      ORIGIN_GEOMETRY          LG          LT  \
331069  POINT (-43.935184 -19.858585)  -43.935184  -19.858585   
331070  POINT (-43.960678 -19.966806)  -43.960678  -19.966806   
331071  POINT (-43.935052 -19.938871)  -43.935052  -19.938871   

               HORARIO_GPS    DATA_GPS   time_diff  \
331069 2024-02-16 05:19:07  2023-12-06         NaN   
331070 2024-02-16 14:35:44  2023-

In [81]:
od['HORA_ORIGEM'] = od['DATAHORA_UTILIZACAO'].dt.hour

In [82]:
print(od[od["CARTAO_USUARIO"] == 6850001298748])

       CODIGO_VEICULO CODIGO_LINHA  CARTAO_USUARIO       TIPO_CARTAO  \
331069          10766         5201   6850001298748  VT CARGA A BORDO   
331070          31139         9250   6850001298748  VT CARGA A BORDO   
331071          20695         8102   6850001298748  VT CARGA A BORDO   

       DATAHORA_UTILIZACAO  VALOR_COBRADO             HORARIO        DATA  \
331069 2023-12-06 05:19:34            450 2024-02-16 05:19:34  2023-12-06   
331070 2023-12-06 14:36:05            450 2024-02-16 14:36:05  2023-12-06   
331071 2023-12-06 15:16:33            225 2024-02-16 15:16:33  2023-12-06   

                      ORIGIN_GEOMETRY          LG          LT  \
331069  POINT (-43.935184 -19.858585)  -43.935184  -19.858585   
331070  POINT (-43.960678 -19.966806)  -43.960678  -19.966806   
331071  POINT (-43.935052 -19.938871)  -43.935052  -19.938871   

               HORARIO_GPS    DATA_GPS   time_diff  \
331069 2024-02-16 05:19:07  2023-12-06         NaN   
331070 2024-02-16 14:35:44  2023-

In [83]:
import h3
from shapely.geometry import Point

In [84]:


# Supondo que 'od' é o seu DataFrame e 'ORIGIN_GEOMETRY' e 'DESTINO_GEOMETRY' são do tipo Point

# Extraindo latitude e longitude para origem e destino
od['origin_lat'] = od['ORIGIN_GEOMETRY'].apply(lambda point: point.y if isinstance(point, Point) else None)
od['origin_lon'] = od['ORIGIN_GEOMETRY'].apply(lambda point: point.x if isinstance(point, Point) else None)
od['destino_lat'] = od['DESTINO_GEOMETRY'].apply(lambda point: point.y if isinstance(point, Point) else None)
od['destino_lon'] = od['DESTINO_GEOMETRY'].apply(lambda point: point.x if isinstance(point, Point) else None)

# Definindo o nível de resolução do H3, por exemplo, 7
resolution = 9

# Calculando o índice H3 para origem e destino
od['h3_origin'] = od.apply(lambda row: h3.geo_to_h3(row['origin_lat'], row['origin_lon'], resolution) if pd.notnull(row['origin_lat']) and pd.notnull(row['origin_lon']) else None, axis=1)
od['h3_destino'] = od.apply(lambda row: h3.geo_to_h3(row['destino_lat'], row['destino_lon'], resolution) if pd.notnull(row['destino_lat']) and pd.notnull(row['destino_lon']) else None, axis=1)

# Exibindo o DataFrame resultante
print(od[['ORIGIN_GEOMETRY', 'h3_origin', 'DESTINO_GEOMETRY', 'h3_destino']])



                                          ORIGIN_GEOMETRY        h3_origin  \
0                           POINT (-44.001505 -19.989762)  89a881a4a0bffff   
1                           POINT (-43.957425 -19.929533)  89a88cdb1c7ffff   
2                           POINT (-43.925964 -19.925411)  89a88cdb2abffff   
3                           POINT (-43.931255 -19.932539)  89a88cdb04fffff   
4                           POINT (-43.986357 -19.957528)  89a88cda617ffff   
...                                                   ...              ...   
766959                       POINT (-43.941248 -19.92075)  89a88cdb383ffff   
766960                      POINT (-43.980648 -19.949388)  89a88cda663ffff   
766961                      POINT (-43.929944 -19.918829)  89a88cdb3cbffff   
766962  Point (612401.75051438994705677 7803290.763778...             None   
766963                      POINT (-43.920484 -19.920043)  89a88cdb23bffff   

                                         DESTINO_GEOMETRY      

In [85]:
od_analise_linha = od[(od['h3_origin'].notna()) | (od['h3_destino'].notna())]
od_analise_linha = od_analise_linha.groupby('CODIGO_LINHA').size().reset_index(name='Origem e destinos')

# Exibindo o DataFrame resultante
print(od_analise_linha)

    CODIGO_LINHA  Origem e destinos
0           1000               1182
1            102                  6
2            103                  8
3           1030                940
4            104               3523
..           ...                ...
328        SC03A                378
329        SC04A               1624
330        SC04B                816
331         SD01               4042
332         SD02               5356

[333 rows x 2 columns]


In [86]:
od_analise_linha_melhor = od[(od['h3_origin'].notna()) & (od['h3_destino'].notna())]
od_analise_linha_melhor = od_analise_linha_melhor.groupby('CODIGO_LINHA').size().reset_index(name='Origem e destinos')

# Exibindo o DataFrame resultante
print(od_analise_linha_melhor)

    CODIGO_LINHA  Origem e destinos
0            102                  2
1            103                  5
2           1030                625
3            104               2529
4            105                399
..           ...                ...
267        SC01R               1526
268        SC02A               1505
269        SC03A                293
270        SC04A               1207
271        SC04B                611

[272 rows x 2 columns]


In [87]:
od_analise_linha

,CODIGO_LINHA,Origem e destinos
0,1000,1182
1,102,6
2,103,8
3,1030,940
4,104,3523
...,...,...
328,SC03A,378
329,SC04A,1624
330,SC04B,816
331,SD01,4042


In [88]:
print(od[od['CODIGO_LINHA']== "67"])

       CODIGO_VEICULO CODIGO_LINHA  CARTAO_USUARIO       TIPO_CARTAO  \
7               30605           67   2310000029199        DEFICIENTE   
213             30607           67   2310000042539        DEFICIENTE   
367             20568           67   2310000043114        DEFICIENTE   
384             30607           67   2310000043219        DEFICIENTE   
485             30605           67   2310000043614        DEFICIENTE   
...               ...          ...             ...               ...   
765276          30607           67   6850003886946  VT CARGA A BORDO   
765937          10839           67   6850003889551  VT CARGA A BORDO   
765939          30632           67   6850003889552  VT CARGA A BORDO   
766225          20564           67   6850003889834  VT CARGA A BORDO   
766877          30607           67   6850003890852  VT CARGA A BORDO   

       DATAHORA_UTILIZACAO  VALOR_COBRADO             HORARIO        DATA  \
7      2023-12-06 18:09:15              0 2024-02-16 18:09

In [89]:
od['CARTAO_USUARIO'] = "ID" + od["CARTAO_USUARIO"].astype(str)


In [90]:
od

,CODIGO_VEICULO,CODIGO_LINHA,CARTAO_USUARIO,TIPO_CARTAO,DATAHORA_UTILIZACAO,VALOR_COBRADO,HORARIO,DATA,ORIGIN_GEOMETRY,LG,...,DATA_GPS,time_diff,DESTINO_GEOMETRY,HORA_ORIGEM,origin_lat,origin_lon,destino_lat,destino_lon,h3_origin,h3_destino
0,40805,3054,ID2310000029191,DEFICIENTE,2023-12-06 05:12:53,0,2024-02-16 05:12:53,2023-12-06,POINT (-44.001505 -19.989762),-44.001505,...,2023-12-06,NaN,POINT (-43.923305 -19.800458),5,-19.989762,-44.001505,-19.800458,-43.923305,89a881a4a0bffff,89a88ccb697ffff
1,31111,33,ID2310000029191,DEFICIENTE,2023-12-06 05:54:09,0,2024-02-16 05:54:09,2023-12-06,POINT (-43.957425 -19.929533),-43.957425,...,2023-12-06,41.266667,POINT (-43.925964 -19.925411),5,-19.929533,-43.957425,-19.925411,-43.925964,89a88cdb1c7ffff,89a88cdb2abffff
2,20476,82,ID2310000029191,DEFICIENTE,2023-12-06 13:08:00,0,2024-02-16 13:08:00,2023-12-06,POINT (-43.925964 -19.925411),-43.925964,...,2023-12-06,433.850000,POINT (-43.931255 -19.932539),13,-19.925411,-43.925964,-19.932539,-43.931255,89a88cdb2abffff,89a88cdb04fffff
3,30688,3051,ID2310000029191,DEFICIENTE,2023-12-06 19:15:52,0,2024-02-16 19:15:52,2023-12-06,POINT (-43.931255 -19.932539),-43.931255,...,2023-12-06,367.866667,POINT (-44.001505 -19.989762),19,-19.932539,-43.931255,-19.989762,-44.001505,89a88cdb04fffff,89a881a4a0bffff
4,31080,3052,ID2310000029195,DEFICIENTE,2023-12-06 09:54:48,0,2024-02-16 09:54:48,2023-12-06,POINT (-43.986357 -19.957528),-43.986357,...,2023-12-06,NaN,POINT (-43.986279 -19.957457),9,-19.957528,-43.986357,-19.957457,-43.986279,89a88cda617ffff,89a88cda617ffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766959,30656,2034,ID6850003891202,VT CARGA A BORDO,2023-12-06 18:29:53,450,2024-02-16 18:29:53,2023-12-06,POINT (-43.941248 -19.92075),-43.941248,...,2023-12-06,NaN,POINT (-43.980648 -19.949388),18,-19.920750,-43.941248,-19.949388,-43.980648,89a88cdb383ffff,89a88cda663ffff
766960,40407,9214,ID6850003891202,VT CARGA A BORDO,2023-12-06 19:58:44,225,2024-02-16 19:58:44,2023-12-06,POINT (-43.980648 -19.949388),-43.980648,...,2023-12-06,88.850000,POINT (-43.941248 -19.92075),19,-19.949388,-43.980648,-19.920750,-43.941248,89a88cda663ffff,89a88cdb383ffff
766961,40511,62,ID6850003891205,VT CARGA A BORDO,2023-12-06 16:37:47,450,2024-02-16 16:37:47,2023-12-06,POINT (-43.929944 -19.918829),-43.929944,...,2023-12-06,NaN,Point (612401.75051438994705677 7803290.763778...,16,-19.918829,-43.929944,NaN,NaN,89a88cdb3cbffff,None
766962,300115,3000,ID6850003891205,VT CARGA A BORDO,2023-12-06 17:13:09,0,2024-02-16 17:13:09,2023-12-06,Point (612401.75051438994705677 7803290.763778...,NaN,...,NaN,35.366667,POINT (-43.929944 -19.918829),17,NaN,NaN,-19.918829,-43.929944,None,89a88cdb3cbffff


In [91]:
od.to_csv(r'g:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_HEXAGONO\od.csv', sep =';')

In [92]:
mco = r"G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_ADAPTACAOBOTELHO\Dezembro_2023\20231206\relopemapadecontroleoperacional.csv"

In [93]:
mco_analise_linha = pd.read_csv(mco, sep = ";", encoding= "latin1")
# Renomeando a coluna 'Código Externo Linha' para 'CODIGO_LINHA'
mco_analise_linha = mco_analise_linha.rename(columns={'Código Externo Linha': 'CODIGO_LINHA'})
mco_analise_linha.columns

Index(['Código Operadora', 'Nome Operadora', 'Nome Garagem',
       'Código Interno Linha', 'CODIGO_LINHA', 'Nome Linha', 'Num Terminal',
       'Viagem', 'Código Veículo', 'Numero Veículo', 'Código Equipamento',
       'Numero de Série do Equipamento', 'Sub Linha',
       'Data Hora Início Operação', 'Data Hora Final Operação',
       'Cartão Motorista', 'Cartão Cobrador', 'Catraca Pendente',
       'Catraca Inicial', 'Catraca Final', 'Distância', 'Viagem.1',
       'Orgão Gestor', 'CMP TER SUB', 'Tipo Viagem',
       'Data Hora Saída Terminal', 'Coleta Pendente', 'Passageiros',
       'Inteiras', 'VT', 'VT Integração', 'Gratuidade', 'Passagens',
       'Bilhete Unitário', 'Passagens Integração', 'Estudantes',
       'EStudantes Integração', 'Intervalo Viagem', 'Terminal', 'Data Coleta',
       'Data Hora Início', 'Data Hora Inserção'],
      dtype='object')

In [94]:
# Agrupando e contando
od_analise_linha = od[(od['h3_origin'].notna()) | (od['h3_destino'].notna())].groupby('CODIGO_LINHA').size()


# Convertendo para DataFrame e renomeando a coluna
od_analise_linha = pd.DataFrame(od_analise_linha, columns=['Contagem_ids_OD'])

# Exibindo o resultado
print(od_analise_linha)


              Contagem_ids_OD
CODIGO_LINHA                 
1000                     1182
102                         6
103                         8
1030                      940
104                      3523
...                       ...
SC03A                     378
SC04A                    1624
SC04B                     816
SD01                     4042
SD02                     5356

[333 rows x 1 columns]


In [95]:
# Agrupando por 'Código Externo Linha' e somando as colunas 'Passageiros' e 'Inteiras'
agrupado_soma = mco_analise_linha.groupby('CODIGO_LINHA')[['Passageiros', 'Inteiras']].sum()

# Calculando a divisão da coluna 'Inteiras' pela coluna 'Passageiros'
agrupado_soma['Proporcao_Inteiras'] = agrupado_soma['Inteiras'].div(agrupado_soma['Passageiros'].replace(0, 1))



# Exibindo o DataFrame resultante
print(agrupado_soma)



              Passageiros  Inteiras  Proporcao_Inteiras
CODIGO_LINHA                                           
1000                 2015         0            0.000000
1001                12457         0            0.000000
1002                   17         0            0.000000
101                   380       380            1.000000
102                  1822      1813            0.995060
...                   ...       ...                 ...
SC03A                 471        79            0.167728
SC04A                1994       296            0.148445
SC04B                1089       230            0.211203
SD01                 7050         0            0.000000
SD02                 9895         0            0.000000

[339 rows x 3 columns]


In [96]:
# Realizando o merge entre agrupado_soma e od_analise_linha
analise_final = pd.merge(od_analise_linha, agrupado_soma, on='CODIGO_LINHA')

# Exibindo o DataFrame resultante
print(analise_final)
analise_final.to_excel(r'G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\analise_linha.xlsx')

              Contagem_ids_OD  Passageiros  Inteiras  Proporcao_Inteiras
CODIGO_LINHA                                                            
1000                     1182         2015         0            0.000000
102                         6         1822      1813            0.995060
103                         8         1729      1720            0.994795
1030                      940         1021        54            0.052889
104                      3523         4150       491            0.118313
...                       ...          ...       ...                 ...
SC03A                     378          471        79            0.167728
SC04A                    1624         1994       296            0.148445
SC04B                     816         1089       230            0.211203
SD01                     4042         7050         0            0.000000
SD02                     5356         9895         0            0.000000

[314 rows x 4 columns]


In [97]:
# Realizando o merge entre agrupado_soma e od_analise_linha
analise_final_melhorcenario = pd.merge(od_analise_linha_melhor, agrupado_soma, on='CODIGO_LINHA')

# Exibindo o DataFrame resultante
print(analise_final)
analise_final_melhorcenario.to_excel(r"G:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_HEXAGONO\geral.xlsx")

              Contagem_ids_OD  Passageiros  Inteiras  Proporcao_Inteiras
CODIGO_LINHA                                                            
1000                     1182         2015         0            0.000000
102                         6         1822      1813            0.995060
103                         8         1729      1720            0.994795
1030                      940         1021        54            0.052889
104                      3523         4150       491            0.118313
...                       ...          ...       ...                 ...
SC03A                     378          471        79            0.167728
SC04A                    1624         1994       296            0.148445
SC04B                     816         1089       230            0.211203
SD01                     4042         7050         0            0.000000
SD02                     5356         9895         0            0.000000

[314 rows x 4 columns]


In [226]:
import folium

# Exemplo de identificador H3
h3_index = '89a88ccb697ffff'

# Obter as coordenadas dos vértices do hexágono
hex_boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)

# Converter as coordenadas para um formato adequado para folium (latitude, longitude)
hex_coords = [[coord[1], coord[0]] for coord in hex_boundary]

# Criar um mapa folium centrado no hexágono
mapa = folium.Map(location=hex_coords[0], zoom_start=12)

# Adicionar o polígono do hexágono ao mapa
folium.Polygon(locations=hex_coords, color='blue', fill=True, fill_color='blue').add_to(mapa)

# Mostrar o mapa
mapa.save(r'g:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\05.3.1.2 OD\OD_HEXAGONO\heatmap.html')

In [227]:
hora_especifica = 5
df_filtrado = od[od['HORA_ORIGEM'] == hora_especifica]
df_filtrado = od[od['h3_origin'] == "89a88ccb697ffff"]
df_filtrado

,CODIGO_VEICULO,CODIGO_LINHA,CARTAO_USUARIO,TIPO_CARTAO,DATAHORA_UTILIZACAO,VALOR_COBRADO,HORARIO,DATA,ORIGIN_GEOMETRY,LG,...,DATA_GPS,time_diff,DESTINO_GEOMETRY,HORA_ORIGEM,origin_lat,origin_lon,destino_lat,destino_lon,h3_origin,h3_destino
2749,11104,738,ID2310000148479,DEFICIENTE,2023-12-06 05:15:25,0,2024-02-16 05:15:25,2023-12-06,POINT (-43.923316 -19.800472),-43.923316,...,2023-12-06,NaN,POINT (-43.979887 -19.841333),5,-19.800472,-43.923316,-19.841333,-43.979887,89a88ccb697ffff,89a88cd9d0fffff
5463,11009,738,ID2310000530095,DEFICIENTE,2023-12-06 07:19:11,0,2024-02-16 07:19:11,2023-12-06,POINT (-43.923348 -19.800454),-43.923348,...,2023-12-06,NaN,None,7,-19.800454,-43.923348,NaN,NaN,89a88ccb697ffff,None
7363,11014,738,ID2310000668234,DEFICIENTE,2023-12-06 04:20:07,0,2024-02-16 04:20:07,2023-12-06,POINT (-43.92334 -19.800462),-43.923340,...,2023-12-06,NaN,POINT (-43.958317 -19.815268),4,-19.800462,-43.923340,-19.815268,-43.958317,89a88ccb697ffff,89a88cd9b13ffff
11893,11014,738,ID2310000847841,DEFICIENTE,2023-12-06 07:53:49,0,2024-02-16 07:53:49,2023-12-06,POINT (-43.923314 -19.800443),-43.923314,...,2023-12-06,NaN,NaN,7,-19.800443,-43.923314,NaN,NaN,89a88ccb697ffff,None
23385,11009,738,ID2320000357763,IDOSO,2023-12-06 06:04:26,0,2024-02-16 06:04:26,2023-12-06,POINT (-43.923316 -19.80046),-43.923316,...,2023-12-06,NaN,POINT (-43.990267 -19.776072),6,-19.800460,-43.923316,-19.776072,-43.990267,89a88ccb697ffff,89a88cca2a7ffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748117,11014,738,ID6850003859935,VT CARGA A BORDO,2023-12-06 07:53:37,420,2024-02-16 07:53:37,2023-12-06,POINT (-43.923314 -19.800436),-43.923314,...,2023-12-06,NaN,Point (608066.40717149002011865 7805630.617508...,7,-19.800436,-43.923314,NaN,NaN,89a88ccb697ffff,None
750490,11105,738,ID6850003862937,VT CARGA A BORDO,2023-12-06 04:32:28,420,2024-02-16 04:32:28,2023-12-06,POINT (-43.923287 -19.800447),-43.923287,...,2023-12-06,NaN,POINT (614055.932846912 7799728.66883284),4,-19.800447,-43.923287,NaN,NaN,89a88ccb697ffff,None
759144,11104,738,ID6850003878603,VT CARGA A BORDO,2023-12-06 05:16:21,420,2024-02-16 05:16:21,2023-12-06,POINT (-43.923311 -19.80045),-43.923311,...,2023-12-06,NaN,POINT (-43.94883 -19.921792),5,-19.800450,-43.923311,-19.921792,-43.948830,89a88ccb697ffff,89a88cdb14bffff
761126,11009,738,ID6850003880758,VT CARGA A BORDO,2023-12-06 07:18:27,420,2024-02-16 07:18:27,2023-12-06,POINT (-43.923376 -19.800469),-43.923376,...,2023-12-06,NaN,POINT (-43.944945 -19.80168),7,-19.800469,-43.923376,-19.801680,-43.944945,89a88ccb697ffff,89a88ccb4a3ffff


In [228]:
from folium.plugins import HeatMap
import folium
import h3

# Contar os destinos baseados em índices H3
contagem_destinos = df_filtrado['h3_destino'].value_counts()

# Converter a contagem de destinos para um formato adequado para o mapa de calor
pontos_mapa_calor = [(h3.h3_to_geo(h)[0], h3.h3_to_geo(h)[1], count) for h, count in contagem_destinos.items()]

# Calcular o centro do mapa como a média das localizações dos destinos
latitudes = [h3.h3_to_geo(h)[0] for h in contagem_destinos.keys()]
longitudes = [h3.h3_to_geo(h)[1] for h in contagem_destinos.keys()]
centro_mapa = [sum(latitudes) / len(latitudes), sum(longitudes) / len(longitudes)]

# Criar um mapa centrado nesse ponto
mapa = folium.Map(location=centro_mapa, zoom_start=12)

# Adicionar o mapa de calor ao mapa com um raio menor
HeatMap(pontos_mapa_calor, radius=10).add_to(mapa)  # Ajuste o valor de 'radius' conforme necessário


In [229]:
mapa.save(r'g:\.shortcut-targets-by-id\1KlHFtH5Jvo7J9AF_VBixQOXpFsNmZvhD\05.3. Pesquisa e Ciência dados - GEPDA\05.3.3 Data Science\05.3.3.1. Central de dados\heatmap.html')